# Intro

In [ ]:
!pip install nlp transformers datasets wandb
!apt install git-lfs

     |████████████████████████████████| 1.7 MB 14.5 MB/s 
     |████████████████████████████████| 3.4 MB 68.9 MB/s 
     |████████████████████████████████| 311 kB 74.9 MB/s 
     |████████████████████████████████| 1.7 MB 67.1 MB/s 
     |████████████████████████████████| 243 kB 69.0 MB/s 
     |████████████████████████████████| 67 kB 6.6 MB/s 
     |████████████████████████████████| 3.3 MB 60.8 MB/s 
     |████████████████████████████████| 596 kB 60.5 MB/s 
     |████████████████████████████████| 895 kB 72.7 MB/s 
     |████████████████████████████████| 1.1 MB 60.3 MB/s 
     |████████████████████████████████| 133 kB 81.2 MB/s 
     |████████████████████████████████| 142 kB 70.2 MB/s 
     |████████████████████████████████| 97 kB 9.2 MB/s 
     |████████████████████████████████| 180 kB 69.8 MB/s 
     |████████████████████████████████| 63 kB 2.0 MB/s 
     |████████████████████████████████| 144 kB 74.4 MB/s 
     |████████████████████████████████| 271 kB 74.6 MB/s 
     |██████████████

In [ ]:
from huggingface_hub import notebook_login

notebook_login()

Login successful
Your token has been saved to /root/.huggingface/token
Authenticated through git-credential store but this isn't the helper defined on your machine.
You might have to re-authenticate when pushing to the Hugging Face Hub. Run the following command in your terminal in case you want to set this credential helper as the default

git config --global credential.helper store


In [ ]:
## IMPORTS
import os
import sys
import logging
from dataclasses import dataclass, field
import json
from typing import Dict, List, Optional

import pandas as pd
import numpy as np
from sklearn.metrics import accuracy_score, precision_recall_fscore_support
from tqdm.notebook import tqdm

import torch
from torch import nn
from torch.utils.data import Dataset, DataLoader

import datasets

from transformers import AutoModelForQuestionAnswering, AutoTokenizer
from transformers import default_data_collator, TrainingArguments, Trainer, EvalPrediction, set_seed


In [ ]:
# MONITOR CPU and GPU

os.environ["WANDB_DISABLED"] = "true"

# os.environ["WANDB_DISABLED"] = "false"
# import wandb
# wandb.init()

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# GLOBAL VARIABLES

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

UNKNOWN = "unknown"
BASE_DIR = "/content/drive/MyDrive/Colab Notebooks/seminar/pretrained-finetuning-bert"

MODEL_NAME = "bert-base-uncased"

MY_MLM_MODEL_NAME = f"{MODEL_NAME}-pretrained-mlm-coqa-stories"
MY_QA_MODEL_NAME = f"{MODEL_NAME}-pretrain-finetuned-coqa-falt"
MY_MLM_MODEL_PATH = f"alistvt/{MY_MLM_MODEL_NAME}"

SEED = 7

MAX_LENGTH = 512

In [ ]:
print(device)

LOGS_DIR = os.path.join(BASE_DIR, "logs/")
MODEL_DIR = os.path.join(BASE_DIR, "model/")
OUTPUT_DIR = os.path.join(BASE_DIR, "output/")
TOKENIZER_DIR = os.path.join(BASE_DIR, "tokenizer/")
ANSWERS_DIR = os.path.join(BASE_DIR, "answers/")

DIRECTORIES = [LOGS_DIR, MODEL_DIR, OUTPUT_DIR, TOKENIZER_DIR]

for direc in DIRECTORIES:
    if not os.path.exists(direc):
        os.makedirs(direc)
        print(direc)

cuda


# Prepare Data

In [ ]:
from datasets import load_dataset
coqa = load_dataset("alistvt/coqa-flat")

Downloading:   0%|          | 0.00/1.41k [00:00<?, ?B/s]

Using custom data configuration alistvt--coqa-flat-859610cee67df2fb


  0%|          | 0/2 [00:00<?, ?it/s]

Downloading:   0%|          | 0.00/1.11M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/16.1M [00:00<?, ?B/s]

  0%|          | 0/2 [00:00<?, ?it/s]

Dataset parquet downloaded and prepared to /root/.cache/huggingface/datasets/parquet/alistvt--coqa-flat-859610cee67df2fb/0.0.0/1638526fd0e8d960534e2155dc54fdff8dce73851f21f031d2fb9c2cf757c121. Subsequent calls will reuse this data.


  0%|          | 0/2 [00:00<?, ?it/s]

In [ ]:
coqa['train'].features

{'bad_turn': Value(dtype='bool', id=None),
 'filename': Value(dtype='string', id=None),
 'id': Value(dtype='string', id=None),
 'index': Value(dtype='int64', id=None),
 'input_text': Value(dtype='string', id=None),
 'name': Value(dtype='string', id=None),
 'question': Value(dtype='string', id=None),
 'source': Value(dtype='string', id=None),
 'span_end': Value(dtype='int64', id=None),
 'span_start': Value(dtype='int64', id=None),
 'span_text': Value(dtype='string', id=None),
 'story': Value(dtype='string', id=None),
 'turn_id': Value(dtype='int64', id=None)}

# Preprocessing

In [ ]:
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)

Downloading:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/570 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/226k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/455k [00:00<?, ?B/s]

In [ ]:
def preprocess_function(examples):
    questions = [q.strip() for q in examples["question"]]
    inputs = tokenizer(
        questions,
        examples["story"],
        max_length=MAX_LENGTH,
        truncation="only_second",
        return_offsets_mapping=True,
        padding="max_length",
    )

    offset_mapping = inputs.pop("offset_mapping")
    start_positions = []
    end_positions = []

    for i, offset in enumerate(offset_mapping):
        start_char = examples["span_start"][i]
        end_char = examples["span_start"][i] + len(examples["span_text"][i])
        sequence_ids = inputs.sequence_ids(i)

        # Find the start and end of the context
        idx = 0
        while sequence_ids[idx] != 1:
            idx += 1
        context_start = idx
        while sequence_ids[idx] == 1:
            idx += 1
        context_end = idx - 1

        # If the answer is not fully inside the context, label it (0, 0)
        if offset[context_start][0] > end_char or offset[context_end][1] < start_char:
            start_positions.append(0)
            end_positions.append(0)
        else:
            # Otherwise it's the start and end token positions
            idx = context_start
            while idx <= context_end and offset[idx][0] <= start_char:
                idx += 1
            start_positions.append(idx - 1)

            idx = context_end
            while idx >= context_start and offset[idx][1] >= end_char:
                idx -= 1
            end_positions.append(idx + 1)

    inputs["start_positions"] = start_positions
    inputs["end_positions"] = end_positions
    return inputs


In [ ]:
tokenized_coqa = coqa.map(preprocess_function, batched=True, remove_columns=coqa["train"].column_names)

Loading cached processed dataset at /root/.cache/huggingface/datasets/parquet/alistvt--coqa-flat-859610cee67df2fb/0.0.0/1638526fd0e8d960534e2155dc54fdff8dce73851f21f031d2fb9c2cf757c121/cache-1c0e73945745e6e3.arrow
Loading cached processed dataset at /root/.cache/huggingface/datasets/parquet/alistvt--coqa-flat-859610cee67df2fb/0.0.0/1638526fd0e8d960534e2155dc54fdff8dce73851f21f031d2fb9c2cf757c121/cache-f8fb1cfbe395a971.arrow


# Model

In [ ]:
model = AutoModelForQuestionAnswering.from_pretrained(MY_MLM_MODEL_PATH)
data_collator = default_data_collator

Downloading:   0%|          | 0.00/692 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/418M [00:00<?, ?B/s]

Some weights of the model checkpoint at alistvt/bert-base-uncased-pretrained-mlm-coqa-stories were not used when initializing BertForQuestionAnswering: ['cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.bias', 'cls.predictions.transform.dense.weight']
- This IS expected if you are initializing BertForQuestionAnswering from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForQuestionAnswering from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForQuestionAnswering were not initialized from the model checkpoint at alistvt/ber

# Training script

In [ ]:
model = model.to(device)
# set_seed(SEED)

In [40]:
training_args = TrainingArguments(
    MY_QA_MODEL_NAME,
    
    # evaluation_strategy="epoch",
    evaluation_strategy="steps",
    eval_steps=2000,

    learning_rate=1e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,

    save_steps=2000,
    load_best_model_at_end=True,

    num_train_epochs=4,

    weight_decay=0.01,
    push_to_hub=True
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_coqa['train'],
    eval_dataset=tokenized_coqa['validation'],
    data_collator=data_collator,
    tokenizer=tokenizer,
)

PyTorch: setting up devices
The default value for the training argument `--report_to` will change in v5 (from all installed integrations to none). In v5, you will need to use `--report_to all` to get the same behavior as now. You should start updating your code and make this info disappear :-).
Using the `WAND_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).
/content/bert-base-uncased-pretrain-finetuned-coqa-falt is already a clone of https://huggingface.co/alistvt/bert-base-uncased-pretrain-finetuned-coqa-falt. Make sure you pull the latest changes with `repo.git_pull()`.


In [ ]:
trainer.train(resume_from_checkpoint=True)
# trainer.train()
trainer.push_to_hub(commit_message="finetuining on Coqa 4 epochs.")
# trainer.save_model()

Loading model from bert-base-uncased-pretrain-finetuned-coqa-falt/checkpoint-20000).
***** Running training *****
  Num examples = 108647
  Num Epochs = 4
  Instantaneous batch size per device = 16
  Total train batch size (w. parallel, distributed & accumulation) = 16
  Gradient Accumulation steps = 1
  Total optimization steps = 27164
  Continuing training from checkpoint, will skip to saved global_step
  Continuing training from epoch 2
  Continuing training from global step 20000
  Will skip the first 2 epochs then the first 6418 batches in the first epoch. If this takes a lot of time, you can add the `--ignore_data_skip` flag to your launch command, but you will resume the training on data already seen by your model.


  0%|          | 0/6418 [00:00<?, ?it/s]

Step,Training Loss,Validation Loss
22000,2.223200,2.811974


***** Running Evaluation *****
  Num examples = 7983
  Batch size = 16
Saving model checkpoint to bert-base-uncased-pretrain-finetuned-coqa-falt/checkpoint-22000
Configuration saved in bert-base-uncased-pretrain-finetuned-coqa-falt/checkpoint-22000/config.json
Model weights saved in bert-base-uncased-pretrain-finetuned-coqa-falt/checkpoint-22000/pytorch_model.bin
tokenizer config file saved in bert-base-uncased-pretrain-finetuned-coqa-falt/checkpoint-22000/tokenizer_config.json
Special tokens file saved in bert-base-uncased-pretrain-finetuned-coqa-falt/checkpoint-22000/special_tokens_map.json
tokenizer config file saved in bert-base-uncased-pretrain-finetuned-coqa-falt/tokenizer_config.json
Special tokens file saved in bert-base-uncased-pretrain-finetuned-coqa-falt/special_tokens_map.json


In [ ]:
# Evaluation
results = {}
    
eval_output = trainer.evaluate()

output_eval_file = os.path.join(training_args.output_dir, "eval_results.txt")
with open(output_eval_file, "w") as writer:
    print("***** Eval results *****")
    for key in sorted(eval_output.keys()):
        print("  %s = %s"% (key, str(eval_output[key])))

results.update(eval_output)

print(results)


***** Running Evaluation *****
  Num examples = 7983
  Batch size = 16


***** Eval results *****
  epoch = 2.0
  eval_loss = 2.7381863594055176
  eval_runtime = 144.1709
  eval_samples_per_second = 55.372
  eval_steps_per_second = 3.461
{'eval_loss': 2.7381863594055176, 'eval_runtime': 144.1709, 'eval_samples_per_second': 55.372, 'eval_steps_per_second': 3.461, 'epoch': 2.0}


In [ ]:
tokenizer.save_pretrained(MODEL_DIR)

tokenizer config file saved in /content/drive/MyDrive/Colab Notebooks/seminar/pretrained-finetuning-bert/model/tokenizer_config.json
Special tokens file saved in /content/drive/MyDrive/Colab Notebooks/seminar/pretrained-finetuning-bert/model/special_tokens_map.json


('/content/drive/MyDrive/Colab Notebooks/seminar/pretrained-finetuning-bert/model/tokenizer_config.json',
 '/content/drive/MyDrive/Colab Notebooks/seminar/pretrained-finetuning-bert/model/special_tokens_map.json',
 '/content/drive/MyDrive/Colab Notebooks/seminar/pretrained-finetuning-bert/model/vocab.txt',
 '/content/drive/MyDrive/Colab Notebooks/seminar/pretrained-finetuning-bert/model/added_tokens.json',
 '/content/drive/MyDrive/Colab Notebooks/seminar/pretrained-finetuning-bert/model/tokenizer.json')

In [ ]:
trainer.save_model(output_dir=MODEL_DIR)

Saving model checkpoint to /content/drive/MyDrive/Colab Notebooks/seminar/pretrained-finetuning-bert/model/
Configuration saved in /content/drive/MyDrive/Colab Notebooks/seminar/pretrained-finetuning-bert/model/config.json
Model weights saved in /content/drive/MyDrive/Colab Notebooks/seminar/pretrained-finetuning-bert/model/pytorch_model.bin
tokenizer config file saved in /content/drive/MyDrive/Colab Notebooks/seminar/pretrained-finetuning-bert/model/tokenizer_config.json
Special tokens file saved in /content/drive/MyDrive/Colab Notebooks/seminar/pretrained-finetuning-bert/model/special_tokens_map.json


In [ ]:
trainer.push_to_hub(MODEL_DIR)

Saving model checkpoint to bert-base-uncased-pretrained-mlm-coqa-stories-pretrain-finetuned-coqa-falttened
Configuration saved in bert-base-uncased-pretrained-mlm-coqa-stories-pretrain-finetuned-coqa-falttened/config.json
Model weights saved in bert-base-uncased-pretrained-mlm-coqa-stories-pretrain-finetuned-coqa-falttened/pytorch_model.bin
tokenizer config file saved in bert-base-uncased-pretrained-mlm-coqa-stories-pretrain-finetuned-coqa-falttened/tokenizer_config.json
Special tokens file saved in bert-base-uncased-pretrained-mlm-coqa-stories-pretrain-finetuned-coqa-falttened/special_tokens_map.json


AttributeError: ignored

# Test model with data

In [ ]:
def construct_answer(tokenizer, model, outputs, encoding):
    start_logits = outputs.start_logits
    end_logits = outputs.end_logits
    all_tokens = tokenizer.convert_ids_to_tokens(encoding["input_ids"][0].tolist())

    start_index = torch.argmax(start_logits)

    end_sorted = torch.argsort(end_logits, descending=True).squeeze().tolist()
    for i in end_sorted:
        if i+1 > start_index:
            end_index = i+1
            break

    if start_index < end_index:
        answer_tokens = all_tokens[start_index:end_index]
        answer = tokenizer.decode(tokenizer.convert_tokens_to_ids(answer_tokens))
    else: # TODO: a good condition for unknown
        answer = UNKNOWN
    return answer

In [ ]:
def test_construct_answer():
    question, text = "who is Ali?", "Ali is a good student."

    encoding = tokenizer(question, text, return_tensors="pt").to(device)

    outputs = model(**encoding)
    # outputs = model(**encoding, start_positions=start_positions, end_positions=end_positions)

    answer = construct_answer(tokenizer, model, outputs, encoding)
    print(answer)

test_construct_answer()

ali is a good student


In [ ]:
df_test = pd.read_pickle("/content/drive/MyDrive/Colab Notebooks/seminar/dataset/coqa_flat_val_df.pkl")

predictions = []

for index, item in tqdm(df_test.iterrows()):
    try:
        question, text = item["question"], item["story"]

        encoding = tokenizer(question, text, return_tensors="pt").to(device)

        outputs = model(**encoding)
        # outputs = model(**encoding, start_positions=start_positions, end_positions=end_positions)

        answer = construct_answer(tokenizer, model, outputs, encoding)

        predictions.append(
            {
                "id": item["id"],
                "turn_id": item["turn_id"],
                "answer": answer
            }
        )
    except Exception as e:
        print(e)
        predictions.append(
            {
                "id": item["id"],
                "turn_id": item["turn_id"],
                "answer": UNKNOWN
            }
        )

0it [00:00, ?it/s]

Token indices sequence length is longer than the specified maximum sequence length for this model (515 > 512). Running this sequence through the model will result in indexing errors


The size of tensor a (515) must match the size of tensor b (512) at non-singleton dimension 1
The size of tensor a (514) must match the size of tensor b (512) at non-singleton dimension 1
The size of tensor a (514) must match the size of tensor b (512) at non-singleton dimension 1
The size of tensor a (801) must match the size of tensor b (512) at non-singleton dimension 1
The size of tensor a (805) must match the size of tensor b (512) at non-singleton dimension 1
The size of tensor a (802) must match the size of tensor b (512) at non-singleton dimension 1
The size of tensor a (801) must match the size of tensor b (512) at non-singleton dimension 1
The size of tensor a (804) must match the size of tensor b (512) at non-singleton dimension 1
The size of tensor a (800) must match the size of tensor b (512) at non-singleton dimension 1
The size of tensor a (809) must match the size of tensor b (512) at non-singleton dimension 1
The size of tensor a (804) must match the size of tensor b (

In [ ]:
with open(OUTPUT_DIR + "answers.json", 'w') as f:
    f.write(json.dumps(predictions))